In [1]:
import sqlite3
import telebot
from telebot import types
from exif import Image
import math
import os
import torch
import clip
from PIL import Image as Image_PIL 

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

In [2]:
def clean_repair(img, message, device=device, model=model, preprocess=preprocess):
    """
        returns flag: if flag equals 0 then car clean and not under repair, else if flag = 1 user have to make another photo of it
    """
    image = preprocess(img).unsqueeze(0).to(device)
    text = clip.tokenize(["dirty car", "clean car"]).to(device)
    #plt.imshow(img)
    flag = 0 
    
    with torch.no_grad():
        image_features = model.encode_image(image)
        text_features = model.encode_text(text)

        logits_per_image, logits_per_text = model(image, text)
        probs = logits_per_image.softmax(dim=-1).cpu().numpy()

    print("car clean probs:", probs) # [[dirt, clean]]

    if probs[0][0] > probs[0][1]:
        flag = 1
        bot.reply_to(message, "Ваша машина слишком грязная, пожалуйста, почистите её перед фото")


    text = clip.tokenize(["disassembled car", "the whole car"]).to(device)    
    with torch.no_grad():
        image_features = model.encode_image(image)
        text_features = model.encode_text(text)

        logits_per_image, logits_per_text = model(image, text)
        probs = logits_per_image.softmax(dim=-1).cpu().numpy()
    print("car repair probs:", probs) # [[under repair, in good condition]]

    if probs[0][0] > probs[0][1]:
        bot.reply_to(message, 'Не принимаются фото автомобиля в разобранном виде или в процессе ремонта')
        flag = 1
    return flag


def vin_number(img, message, device=device, model=model, preprocess=preprocess): # ocr=reader
    """
        returns flag: if flag equals 0 then vin number on the car metal body, else if flag = 1 user have to make another photo of it 
        TODO: OCR model for 
    """
    image = preprocess(img).unsqueeze(0).to(device)
    text = clip.tokenize(["VIN number on the car metal body", "not a VIN number", "car"]).to(device)
    #plt.imshow(img)
    flag = 0 
    
    with torch.no_grad():
        image_features = model.encode_image(image)
        text_features = model.encode_text(text)

        logits_per_image, logits_per_text = model(image, text)
        probs = logits_per_image.softmax(dim=-1).cpu().numpy()

    print("vin probs:", probs) # [[vin on a metal body, not vin]]

    if probs[0][1] > probs[0][0] or probs[0][2] > probs[0][0]:
        flag = 1
        bot.reply_to(message, 'Требуется фотография VIN на металле. Повторите снова.')
#     else:
#         vin = ocr.readtxt(img)
#         for res in vin:
#             vin_len = len(res[1])
#             print('VIN:', res[1])
#             print('VIN length:', vin_len)
#             if (vil_len != 17):
#                 flag = 1
#                 print('на фотографии не разобрать VIN')
#             else:  # вдруг на металле выбиты ещё символы
#                 flag = 0
        

## тут стоило бы прикрутить ocr
    return flag    


def car_outside_photo(img, message, device=device, model=model, preprocess=preprocess):
    """
        returns flag: if flag equals 0 then its a car photo, else if flag = 1 looks like its not a real car or not outside
    """
    image = preprocess(img).unsqueeze(0).to(device)
    text = clip.tokenize(["car or car outside", "drawing of a car", "car interior"]).to(device)
    #plt.imshow(img)
    flag = 0 
    
    with torch.no_grad():
        image_features = model.encode_image(image)
        text_features = model.encode_text(text)

        logits_per_image, logits_per_text = model(image, text)
        probs = logits_per_image.softmax(dim=-1).cpu().numpy()

    print("car outside probs:", probs) 

    if probs[0][1] > probs[0][0]:
        flag = 1
        bot.reply_to(message, 'Требуются фотографии настоящей машины. Повторите снова.')
    elif probs[0][2] > probs[0][0]:
        flag = 1
        bot.reply_to(message, 'Требуются фотографии машины снаружи, заполняющей большую часть кадра. Повторите снова.')    

    return flag  


def car_windshield(img, message, device=device, model=model, preprocess=preprocess):
    """
        returns flag: if flag equals 0 then its a windshield of a car, else if flag = 1 then its not
    """
    image = preprocess(img).unsqueeze(0).to(device)
    text = clip.tokenize(["car windshield close photo", "windshield drawing", "photo of the whole car", "not a car"]).to(device)
    #plt.imshow(img)
    flag = 0 
    
    with torch.no_grad():
        image_features = model.encode_image(image)
        text_features = model.encode_text(text)

        logits_per_image, logits_per_text = model(image, text)
        probs = logits_per_image.softmax(dim=-1).cpu().numpy()

    print("windshield probs:", probs) 

    if probs[0][1] > probs[0][0] or probs[0][2] > probs[0][0] or probs[0][3] > probs[0][0]:
        flag = 1
        bot.reply_to(message, 'Требуется фотография лобового стекла настоящего автомобиля крупным планом. Повторите снова.') 
        

    return flag  


def car_windshield_markings(img, message, device=device, model=model, preprocess=preprocess):
    """
        returns flag: if flag equals 0 then its a windshield markings of a car, else if flag = 1 then its not
    """
    image = preprocess(img).unsqueeze(0).to(device)
    text = clip.tokenize(["identification label on a real car windshield", "markings or identification label not on a car windshield", "windshield without markings"]).to(device)
    #plt.imshow(img)
    flag = 0 
    
    with torch.no_grad():
        image_features = model.encode_image(image)
        text_features = model.encode_text(text)

        logits_per_image, logits_per_text = model(image, text)
        probs = logits_per_image.softmax(dim=-1).cpu().numpy()

    print("windshield markings probs:", probs) 

    if probs[0][1] > probs[0][0] or probs[0][2] > probs[0][0]:
        flag = 1
        bot.reply_to(message, 'Требуется фотография маркировки на лобовом стекле. Повторите снова.') 
        

    return flag  


def car_wheel(img, message, device=device, model=model, preprocess=preprocess):
    """
        returns flag: if flag equals 0 then its ok, else if flag = 1 then its not
    """
    image = preprocess(img).unsqueeze(0).to(device)
    text = clip.tokenize(["photo of the assembled wheel with an inscription about the size and manufacturer", "photo of the disassembled wheel", "wheel without an inscription or not a wheel"]).to(device)
    #plt.imshow(img)
    flag = 0 
    
    with torch.no_grad():
        image_features = model.encode_image(image)
        text_features = model.encode_text(text)

        logits_per_image, logits_per_text = model(image, text)
        probs = logits_per_image.softmax(dim=-1).cpu().numpy()

    print("wheel probs:", probs) 

    if probs[0][1] > probs[0][0] or probs[0][2] > probs[0][0]:
        flag = 1
        bot.reply_to(message, 'Требуется фотография колеса в сборе, должны читаться размер и производитель шин. Повторите снова.') 
        

    return flag  


def mileage_odometer(img, message, device=device, model=model, preprocess=preprocess):
    """
        returns flag: if flag equals 0 then its ok, else if flag = 1 then its not
    """
    image = preprocess(img).unsqueeze(0).to(device)
    text = clip.tokenize(["photo of an odometer with mileage", "odometer without mileage numbers", "not an odometer"]).to(device)
    #plt.imshow(img)
    flag = 0 
    
    with torch.no_grad():
        image_features = model.encode_image(image)
        text_features = model.encode_text(text)

        logits_per_image, logits_per_text = model(image, text)
        probs = logits_per_image.softmax(dim=-1).cpu().numpy()

    print("odometer probs:", probs) 

    if probs[0][1] > probs[0][0] or probs[0][2] > probs[0][0]:
        flag = 1
        bot.reply_to(message, 'Требуется фотография одометра, где виден пробег. Повторите снова.') 
        

    return flag 


def interior_front(img, message, device=device, model=model, preprocess=preprocess):
    """
        returns flag: if flag equals 0 then its ok, else if flag = 1 then its not
    """
    image = preprocess(img).unsqueeze(0).to(device)
    text = clip.tokenize(["car interior front part", "not front part of car interior, or not interior", "car interior back seats"]).to(device)
    #plt.imshow(img)
    flag = 0 
    
    with torch.no_grad():
        image_features = model.encode_image(image)
        text_features = model.encode_text(text)

        logits_per_image, logits_per_text = model(image, text)
        probs = logits_per_image.softmax(dim=-1).cpu().numpy()

    print("interior front probs:", probs) 

    if probs[0][1] > probs[0][0] or probs[0][2] > probs[0][0]:
        flag = 1
        bot.reply_to(message, 'Требуется фотография передней части салона автомобиля с приборной панелью. Повторите снова.') 
        

    return flag


def interior_back(img, message, device=device, model=model, preprocess=preprocess):
    """
        returns flag: if flag equals 0 then its ok, else if flag = 1 then its not
    """
    image = preprocess(img).unsqueeze(0).to(device)
    text = clip.tokenize(["car interior back part with seats", "car interior front part with panel", "not car or car exterior"]).to(device)
    #plt.imshow(img)
    flag = 0 
    
    with torch.no_grad():
        image_features = model.encode_image(image)
        text_features = model.encode_text(text)

        logits_per_image, logits_per_text = model(image, text)
        probs = logits_per_image.softmax(dim=-1).cpu().numpy()

    print("interior back probs:", probs) 

    if probs[0][1] > probs[0][0] or probs[0][2] > probs[0][0]:
        flag = 1
        bot.reply_to(message, 'Требуется фотография задней части салона автомобиля. Повторите снова.') 
        

    return flag  


def car_damage(img, message, device=device, model=model, preprocess=preprocess):
    """
        returns flag: if flag equals 0 then its ok, else if flag = 1 then its not
    """
    image = preprocess(img).unsqueeze(0).to(device)
    text = clip.tokenize(["severe damaged car", "scratches on the car", "car dent", "car is not damaged"]).to(device)
    #plt.imshow(img)
#     flag = 0 
    
    with torch.no_grad():
        image_features = model.encode_image(image)
        text_features = model.encode_text(text)

        logits_per_image, logits_per_text = model(image, text)
        probs = logits_per_image.softmax(dim=-1).cpu().numpy()
    print("damage probs:", probs) 
    

    maxprob = probs[0].argmax()
    
    if probs[0][0] > 0.4:
        print('Сильные повреждения')
    if probs[0][1] > 0.4:
        print('Царапины')   
    if probs[0][2] > 0.4:
        print('Вмятины')       
        

#     return flag  


def car_keys(img, message, device=device, model=model, preprocess=preprocess):
    """
        returns flag: if flag equals 0 then its ok, else if flag = 1 then its not
    """
    image = preprocess(img).unsqueeze(0).to(device)
    text = clip.tokenize(["car keys", "car alarm keychain", "the keys for house", "not keys", "car"]).to(device)
    #plt.imshow(img)
    flag = 0 
    
    with torch.no_grad():
        image_features = model.encode_image(image)
        text_features = model.encode_text(text)

        logits_per_image, logits_per_text = model(image, text)
        probs = logits_per_image.softmax(dim=-1).cpu().numpy()
    print("key probs:", probs) 
    

    maxprob = probs[0].argmax()
    
    if maxprob == 0 or maxprob == 1:
        print('Ключи от машины или брелок сигнализации')  
    if maxprob == 2 or maxprob == 3 or maxprob == 4:
        bot.reply_to(message, 'На фотографии что-то другое')     
        flag = 1
    return flag  

In [3]:
def gps_to_decimal(gps):
    return gps[0] + (gps[1] / 60.0) + (gps[2] / 3600.0)


def distance_on_unit_sphere(lat1, long1, lat2, long2):
    # перевод в радианы
    lat1 = math.radians(lat1)
    long1 = math.radians(long1)
    lat2 = math.radians(lat2)
    long2 = math.radians(long2)

    # формула гаверсинусов
    dlat = lat2 - lat1
    dlong = long2 - long1
    a = (math.sin(dlat / 2) ** 2) + math.cos(lat1) * math.cos(lat2) * (math.sin(dlong / 2) ** 2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    # Радиус Земли в километрах
    radius = 6371

    # Расстояние на поверхности Земли
    distance = radius * c
    return distance * 1000


latitude_first, longtitude_first = [], []
date_first = ''
def check_requirements(path, state, message):
    global latitude_first
    global longtitude_first
    global date_first
    with open(path, "rb") as file:
        image = Image(file)
        if image.has_exif:

            if image.pixel_x_dimension <= 1600 and image.pixel_y_dimension <= 1200:
                bot.reply_to(message, "У данной фотографии маленькое разрешение")
                bot.reply_to(message, "Повторите снова")
                os.remove(path)
                return 0

            if state == "vin":
                date_first = image.datetime[:-9]
                latitude_first, longtitude_first = gps_to_decimal(image.gps_latitude), gps_to_decimal(image.gps_longitude)

            latitude_current, longtitude_current = gps_to_decimal(image.gps_latitude), gps_to_decimal(image.gps_longitude)
            distance = distance_on_unit_sphere(latitude_first, longtitude_first, latitude_current, longtitude_current)
            if  distance >= 100. :
                bot.reply_to(message, "Одно фото с Таганрога, а другое с Камчатки, почему...")
                bot.reply_to(message, "Повторите снова")
                os.remove(path)
                return 0
            
            if date_first!=image.datetime[:-9]:
                bot.reply_to(message, "Фотографии должны быть сделаны в один день")
                bot.reply_to(message, "Повторите снова")
                os.remove(path)
                return 0

            else:
                bot.reply_to(message, "У данной фотографии не найдены метаданные")
                bot.reply_to(message, "Повторите снова")
                os.remove(path)
                return 0 
    return 1


In [4]:
def car_checking(state, path, message):
    global continue_state
    if check_requirements(path, state, message):
        img = Image_PIL.open(path)
        if state=='vin':
            car_requirement = vin_number(img, message)
            if car_requirement==0 and len(os.listdir(newpath2 + "/" + state))==1:
                user_state[message.chat.id] = 'car8'
                bot.reply_to(message, "Фото подходит всем требованиям")
                bot.reply_to(message, "Фото VIN-номера добавлено")
                bot.send_message(message.chat.id, 'Отправьте фото машины с четырех углов и четырех сторон')
#                 bot.send_message(message.chat.id, 'Перед отправкой проверьте, чтобы фотографии были файлами c расширением *.jpg')
            elif car_requirement==1:
                os.remove(path)
                     
        elif state == 'car8':
            car_requirement = car_outside_photo(img, message) or clean_repair(img, message)
            if car_requirement == 0 and len(os.listdir(newpath2 + "/" + state))==8:
                user_state[message.chat.id] = 'windshield'
                bot.reply_to(message, "Фото подходит всем требованиям")
                bot.reply_to(message, "Фото машины с 4 сторон и 4 углов добавлены")
                bot.send_message(message.chat.id, 'Отправьте сперва фото лобового стекла, затем фото его маркировки')
#                 bot.send_message(message.chat.id, 'Перед отправкой проверьте, чтобы фотографии были файлами c расширением *.jpg')
            elif car_requirement==1:
                os.remove(path)
                
        elif state == 'windshield':
            if len(os.listdir(newpath2 + "/" + state))==1:
                car_requirement = car_windshield(img, message) 
            elif len(os.listdir(newpath2 + "/" + state))==2:
                car_requirement = car_windshield_markings(img, message)                 
            if car_requirement == 0 and len(os.listdir(newpath2 + "/" + state))==2:
                user_state[message.chat.id] = 'wheels'
                bot.reply_to(message, "Фото подходит всем требованиям")
                bot.reply_to(message, "Фото лобового стекла и его маркировки добавлены")
                bot.send_message(message.chat.id, 'Отправьте фото колеса в сборе')
#                 bot.send_message(message.chat.id, 'Перед отправкой проверьте, чтобы фотографии были файлами c расширением *.jpg')
            elif car_requirement==1:
                os.remove(path)
                
        elif state == 'wheels':            
            car_requirement = car_wheel(img, message) 
            if car_requirement == 0 and len(os.listdir(newpath2 + "/" + state))==1:
                user_state[message.chat.id] = 'odometer'
                bot.reply_to(message, "Фото подходит всем требованиям")
                bot.reply_to(message, "Фото колеса в сборе добавлено")
                bot.send_message(message.chat.id, 'Отправьте фото показания одометра (пробег)')
#                 bot.send_message(message.chat.id, 'Перед отправкой проверьте, чтобы фотографии были файлами c расширением *.jpg')
            elif car_requirement==1:
                os.remove(path)
                                    
        elif state == 'odometer':
            car_requirement = mileage_odometer(img, message) 
            if car_requirement == 0 and len(os.listdir(newpath2 + "/" + state))==1:
                user_state[message.chat.id] = 'car_interior'
                bot.reply_to(message, "Фото подходит всем требованиям")
                bot.reply_to(message, "Фото показаний одометра добавлены")
                bot.send_message(message.chat.id, 'Отправьте сперва фото передней части салона с приборной панелью, затем фото задней части салона')
#                 bot.send_message(message.chat.id, 'Перед отправкой проверьте, чтобы фотографии были файлами c расширением *.jpg')
            elif car_requirement==1:
                os.remove(path)
                
        elif state == 'car_interior':
            if len(os.listdir(newpath2 + "/" + state))==1:
                car_requirement = interior_front(img, message) 
            elif len(os.listdir(newpath2 + "/" + state))==2:
                car_requirement = interior_back(img, message) 
            if car_requirement == 0 and len(os.listdir(newpath2 + "/" + state))==2:
                user_state[message.chat.id] = 'damage'
                bot.reply_to(message, "Фото подходит всем требованиям")
                bot.reply_to(message, "Фото салона добавлены")
                bot.send_message(message.chat.id, 'Отправьте фото всех повреждений (при наличии)')
#                 bot.send_message(message.chat.id, 'Перед отправкой проверьте, чтобы фотографии были файлами c расширением *.jpg')
                markup=types.ReplyKeyboardMarkup(resize_keyboard=True)
                item=types.KeyboardButton("Далее")
                markup.add(item)
                bot.send_message(message.chat.id, 'Если повреждений нет или вы отправили все необходимые фото, нажмите кнопку "Далее".', reply_markup=markup)
            elif car_requirement==1:
                os.remove(path)
                
        elif state == 'car_keys_immobilizers':
            car_requirement = car_keys(img, message) 
            if car_requirement==1:
                os.remove(path)
                


In [ ]:
ADMIN_CODE = 1377

bot = telebot.TeleBot("bot-token")

user_state = {}
user_id_current_app = ''

abs_path = os.getcwd()

conn = sqlite3.connect(abs_path +'/Database.db', check_same_thread=False)
cursor = conn.cursor()

conn_app = sqlite3.connect(abs_path + '/applications.db', check_same_thread=False)
cursor_app = conn_app.cursor()
cursor_app_folder = conn_app.cursor()

conn_admin = sqlite3.connect(abs_path + '/adminstrators.db', check_same_thread=False)
cursor_admin = conn_admin.cursor()

def db_table_val(user_id: int, user_name: str, user_surname: str, username: str, phone: str):
    cursor.execute('INSERT INTO Database (user_id, user_name, user_surname, username, phone) VALUES (?, ?, ?, ?, ?)',
                    (user_id, user_name, user_surname, username, phone))
    conn.commit()
    
    
def db_app_val(id_app: str, user_id: int, app_status: str, estate: str):
    cursor_app.execute('INSERT INTO Application (id_app, user_id, estate, app_status) VALUES (?, ?, ?, ?)',
                    (id_app, user_id, estate, app_status))
    conn_app.commit()

def db_admin_val(user_id: int, user_name: str, user_surname: str):
    cursor_admin.execute('INSERT INTO adminstrators (user_id, user_name, user_surname) VALUES (?, ?, ?)',
                    (user_id, user_name, user_surname))
    conn_admin.commit()

registered_users = {}


@bot.message_handler(commands=['start'])
def send_welcome(message):
#     bot.reply_to(message, "Здравствуйте! Чтобы зарегистрироваться, используй команду /register.")
    markup=types.ReplyKeyboardMarkup(resize_keyboard=True)
    item1 = types.KeyboardButton("Регистрация пользователя")
    item2 = types.KeyboardButton("Регистрация администратора")
    markup.add(item1)
    markup.add(item2)
#     hideBoard = types.ReplyKeyboardRemove()
    msg = bot.send_message(message.chat.id, 
                           'Здравствуйте! Нам нужно авторизовать Вас, выберите пользователь Вы или Администратор и нажмите соответствующую кнопку.',
                           reply_markup=markup)


###ADMIN###
@bot.message_handler(func=lambda message: message.text.lower() == "регистрация администратора")
def register_admin_menu(message):
    hideBoard = types.ReplyKeyboardRemove()

    # проверяем что пользователь в таблице
    cursor_admin.execute(f"SELECT user_id FROM adminstrators WHERE `user_id` = {message.from_user.id}") # 907423303
    # Получаем результат сделанного запроса
    results = cursor_admin.fetchall()
    if results != []:
        msg = bot.send_message(message.chat.id, "Вы уже есть в системе, ожидайте заявок!", reply_markup=hideBoard)
#         bot.register_next_step_handler(msg, estate_choice)
    else:
        bot.send_message(message.chat.id, "Вас еще нет в системе, давайте зарегистрируемся!", reply_markup=hideBoard)
        # Отправляем сообщение с просьбой о вводе имени 
        msg = bot.send_message(message.chat.id, 'Введите код доступа')
        bot.register_next_step_handler(msg, register_admin)


def register_admin(message):
    # Отправляем сообщение с просьбой о вводе имени
#     bot.send_message(message.chat.id, 'Добро пожаловать, администратор!')
    if int(message.text) == ADMIN_CODE:
        msg = bot.send_message(message.chat.id, 'Код верный! Введите ваше имя')
        bot.register_next_step_handler(msg, process_first_name_admin)     
    else: 
        bot.send_message(message.chat.id, 'Код не верный! У Вас нет прав админа.')        
        
        
fn_admin = 0
# Обработчик ввода имени
def process_first_name_admin(message):
    global fn_admin
    try:
        # Сохраняем имя в переменную
        first_name = message.text
        fn_admin = first_name
        # Отправляем сообщение с просьбой о вводе фамилии
        msg = bot.send_message(message.chat.id, "Введите вашу фамилию")
        bot.register_next_step_handler(msg, process_last_name_admin)

    except Exception as e:
        bot.reply_to(message, e)

# Обработчик ввода фамилии
def process_last_name_admin(message):
    try:
        # Сохраняем фамилию в переменную
        ln_admin = message.text
        us_id = message.from_user.id
        db_admin_val(user_id=us_id, user_name=fn_admin, user_surname=ln_admin)
        bot.reply_to(message, "Вы успешно зарегистрировались как админ!")

    except Exception as e:
        bot.reply_to(message, e)

        
###CLIENT###
@bot.message_handler(func=lambda message: message.text.lower() == "регистрация пользователя")
def check_register(message): 
    hideBoard1 = types.ReplyKeyboardMarkup(resize_keyboard=True)
    item1 = types.KeyboardButton("Создать новую заявку")
    item2 = types.KeyboardButton("Проверить статус существующей заявки")
    hideBoard1.add(item1)
    hideBoard1.add(item2)
    
    hideBoard2 = types.ReplyKeyboardRemove()
    # проверяем что пользователь в таблице
    cursor.execute(f"SELECT user_id FROM Database WHERE `user_id` = {message.from_user.id}") # 907423303
    # Получаем результат сделанного запроса
    results = cursor.fetchall()
    if results != []:
        msg = bot.send_message(message.chat.id, "Вы уже есть в системе, можете воспользоваться функционалом!", reply_markup=hideBoard1)
        bot.register_next_step_handler(msg, estate_choice)
    else:
        msg = bot.send_message(message.chat.id, "Вас еще нет в системе, давайте зарегистрируемся!", reply_markup=hideBoard2)
        register(msg)


# @bot.message_handler(commands=['register'])
def register(message):
    # Отправляем сообщение с просьбой о вводе имени
    msg = bot.send_message(message.chat.id, "Введите ваше имя")
    bot.register_next_step_handler(msg, process_first_name)

    
ln = ''
fn = ''
# Обработчик ввода имени
def process_first_name(message):
    global fn
    try:
        # Сохраняем имя в переменную
        first_name = message.text
        fn = first_name
        # Отправляем сообщение с просьбой о вводе фамилии
        msg = bot.send_message(message.chat.id, "Введите вашу фамилию")
        bot.register_next_step_handler(msg, process_last_name)

    except Exception as e:
        bot.reply_to(message, e)

        
# Обработчик ввода фамилии
def process_last_name(message):
    global ln
    try:
        # Сохраняем фамилию в переменную
        last_name = message.text
        ln = last_name
        # Отправляем сообщение с просьбой о вводе номера телефона
        msg = bot.send_message(message.chat.id, "Введите ваш номер телефона")
        bot.register_next_step_handler(msg, process_phone_number)

    except Exception as e:
        bot.reply_to(message, e)

        
# Обработчик ввода номера телефона
newpath = []
def process_phone_number(message):
    try:
        # Сохраняем номер телефона в переменную
        phone_number = message.text
        us_id = message.from_user.id
        username = message.from_user.username
        db_table_val(user_id=us_id, user_name=fn, user_surname=ln, username=username, phone=phone_number)

        
        hideBoard1 = types.ReplyKeyboardMarkup(resize_keyboard=True)
        item1 = types.KeyboardButton("Создать новую заявку")
        item2 = types.KeyboardButton("Проверить статус существующей заявки")
        hideBoard1.add(item1)
        hideBoard1.add(item2)
        # Отправляем сообщение с подтверждением регистрации
        msg = bot.reply_to(message, "Вы успешно зарегистрированы, можете воспользоваться функционалом!", reply_markup=hideBoard1)

        bot.register_next_step_handler(msg, estate_choice)
    except Exception as e:
        bot.reply_to(message, e)


def estate_choice(message):
    global newpath
    newpath = abs_path + "/" + str(message.from_user.id)
    print(newpath)
    if not os.path.exists(newpath):
        os.makedirs(newpath)
    if message.text == "Создать новую заявку":
        markup=types.ReplyKeyboardMarkup(resize_keyboard=True)
        item1=types.KeyboardButton("Машина")
        item2 = types.KeyboardButton("Дом")
        markup.add(item1)
        markup.add(item2)
        hideBoard = types.ReplyKeyboardRemove()
        msg = bot.send_message(message.chat.id,'Выберите, что хотите застраховать',reply_markup=markup)
        bot.register_next_step_handler(msg, estate_choice_reply)
        
    if message.text.lower() == "проверить статус существующей заявки":
        cursor_check = conn_app.cursor()
        cursor_check.execute("SELECT app_status FROM Application WHERE user_id = ?", (message.chat.id,))
        results = cursor_check.fetchall() 
        if len(results)!=0:
            bot.reply_to(message, f'Статус заявки: {results[-1][0]}')  
        else:
            bot.reply_to(message, 'Вы ещё не подавали заявку.') 
            check_register(message)


estate_name = ""
def estate_choice_reply(message):
    global estate_name
    if message.text == "Машина":
        estate_name = message.text
        markup_cancel=types.ReplyKeyboardMarkup(resize_keyboard=True)
        item = types.KeyboardButton("Назад")
        markup_cancel.add(item)
        msg = bot.send_message(message.chat.id, 'Пришлите описание объекта', reply_markup=markup_cancel)
        bot.register_next_step_handler(msg, describtion)

    elif message.text == "Дом":
        estate_name = message.text
        pass
        # то же самое что и для машины

newpath2 = ""        


def new_app(message):
    global user_id_current_app
    #conn_app = sqlite3.connect('/usr/src/app/bot/dasha/applications.db')
    cursor_process = conn_app.cursor()
    
    cursor_process.execute(f"UPDATE Application SET app_status = 'В процессе' WHERE user_id = {user_id_current_app}")
    conn_app.commit()
    
    markup=types.ReplyKeyboardMarkup(resize_keyboard=True)
    item1=types.KeyboardButton("Принять")
    item2 = types.KeyboardButton("Отклонить")
    markup.add(item1)
    markup.add(item2)
    hideBoard = types.ReplyKeyboardRemove()
    msg = bot.send_message(message.chat.id,'Что сделать с заявкой',reply_markup=markup)
    bot.register_next_step_handler(msg, admission_handling)
    
def admission_handling(message):
    global user_id_current_app
    try:    
        if message.text == "Принять":
            
            #conn_app = sqlite3.connect('/usr/src/app/bot/dasha/applications.db')
            cursor_accept = conn_app.cursor()
            
            cursor_accept.execute(f"UPDATE Application SET app_status = 'Заявка принята' WHERE user_id = {user_id_current_app}")
            conn_app.commit()
            
        elif message.text == "Отклонить":
            
            #conn_app = sqlite3.connect('/usr/src/app/bot/dasha/applications.db')
            cursor_decline = conn_app.cursor()
            
            cursor_decline.execute(f"UPDATE Application SET app_status = 'Заявка отклонена' WHERE user_id = {user_id_current_app}")
            conn_decline.commit()
        
    except Exception as e:
        bot.reply_to(message, e)
        
        
@bot.message_handler(commands=['check_application'])
def check_application(message):
    global user_id_current_app
    #conn_app = sqlite3.connect('/usr/src/app/bot/dasha/applications.db')
    cursor_check = conn_app.cursor()
    cursor_check.execute("SELECT app_status FROM Application WHERE user_id = ?", (user_id_current_app,))
    new_app_status = cursor_check.fetchone()[0]    
    # Отправляем подтверждение пользователю и новое значение app_status
    bot.reply_to(message, f'Статус заявки: {new_app_status}')
            
        
@bot.message_handler(content_types=['text'])
def describtion(message):
    global newpath2
    if message.text.lower()=='назад':
        message.text = "Создать новую заявку"
        estate_choice(message)
        
    elif len(message.text)>1:
        print(newpath)
        app_id = str(message.from_user.id) + "_" + str(len(os.listdir(newpath))+1)
        us_id = message.from_user.id
        status = 'Прием фотографий'
        
        db_app_val(id_app=app_id, user_id=us_id, estate=estate_name, app_status=status)
        newpath2 = newpath + "/" + app_id
        if not os.path.exists(newpath2):
            os.makedirs(newpath2)
        user_text = message.text
        bot.reply_to(message, f"Вы написали: {user_text}")
        with open(newpath2 + "/" + 'description.txt', "w+") as f:
            f.write(user_text)
        bot.send_message(message.chat.id, 'Ваше описание было сохранено')
        bot.send_message(message.chat.id, "Теперь Вам нужно отправить фотографии. Убедитесь что фотографии соответствуют критериям: \n    1) фото загружено как файл \n    2) фото имеет разрешение '.jpg'")
        bot.send_message(message.chat.id, 'Отправьте фото VIN-номера на металле')
        user_state[message.chat.id] = 'vin'
    
def skip_state(message):
    global user_id_current_app
    if message.text == 'Далее':
        path_state = newpath2 + "/" + user_state[message.chat.id]
        if user_state[message.chat.id]=='damage':
            if os.path.exists(path_state):
                if len(os.listdir(path_state))>0:
                    hideBoard = types.ReplyKeyboardRemove()
                    bot.reply_to(message, "Фото подходит всем требованиям", reply_markup=hideBoard)
                    bot.reply_to(message, "Фото всех повреждений добавлены")
            user_state[message.chat.id] = 'car_keys_immobilizers'
            bot.send_message(message.chat.id, 'Отправьте фото штатных ключей + ключей/брелоков/меток от доп. противоугонных устройств')
#             bot.send_message(message.chat.id, 'Перед отправкой проверьте, чтобы фотографии были файлами c расширением *.jpg')
            markup=types.ReplyKeyboardMarkup(resize_keyboard=True)
            item=types.KeyboardButton("Далее")
            markup.add(item)
            bot.send_message(message.chat.id, 'Если ключей/брелоков/меток от доп. противоугонных устройств нет или вы отправили все необходимые фото, нажмите кнопку "Далее".', reply_markup=markup)
        
        elif user_state[message.chat.id]=='car_keys_immobilizers':
            if os.path.exists(path_state):   
                print(len(os.listdir(path_state)))
                if len(os.listdir(path_state))>=1: 
                    hideBoard = types.ReplyKeyboardRemove()
                    bot.reply_to(message, "Фото подходит всем требованиям", reply_markup=hideBoard)
                    if len(os.listdir(path_state))==1:      
                        bot.reply_to(message, "Фото штатных ключей добавлены")
                    elif len(os.listdir(path_state))>1:
                        bot.reply_to(message, "Фото штатных ключей + ключей/брелоков/меток от дополнительных противоугонных устройств добавлены")
                    print('FINAAAL')
                    bot.send_message(message.chat.id, 'Ваша заявка была успешно сохранена!')
                    msg = bot.send_message(468110341, f'Поступила новая заявка от {message.chat.id}, просмотрите фотографии в папке с этим номером.')
                    user_id_current_app = message.chat.id
                    new_app(msg)
                    bot.send_message(message.chat.id, 'Чтобы проверить статус заявки, вызовите команду /check_application')
                    
            else:
                bot.reply_to(message, "Требуется хотя бы одно фото штатных ключей.")
                
@bot.message_handler(content_types=['document'])
def handle_docs_photo(message):

    try:
        file_info = bot.get_file(message.document.file_id)
        downloaded_file = bot.download_file(file_info.file_path)
        state = user_state.get(message.chat.id)
        print(state)
        if not os.path.exists(newpath2 + "/" + state):
            os.makedirs(newpath2 + "/" + state)
        src = newpath2 + "/" + state +"/" + message.document.file_name
        with open(src, 'wb') as new_file:
            new_file.write(downloaded_file)
        if estate_name == 'Машина':
            car_checking(state, src, message)
        elif estate_name == 'Дом':
            house_checking(state, src, message)
                

    except Exception as e:

        bot.reply_to(message, e)


bot.polling(none_stop=True)

/usr/src/app/bot/907423303
/usr/src/app/bot/907423303
/usr/src/app/bot/907423303
